In [36]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

## Variational Autoencoder Architecture

In [5]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [6]:
latent_dim = 2
encoder_inputs = tf.keras.Input(shape = (64, 64, 1))

x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(encoder_inputs)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((3, 1))(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((3, 1))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(16, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(encoder_inputs, z, name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   448         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   24640       leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [7]:
latent_inputs = tf.keras.Input(shape=(latent_dim,))
y = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
y = layers.Reshape((64, 64, 64))(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), padding="same")(y)
y = tf.keras.layers.LeakyReLU()(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), padding="same")(y)
y = tf.keras.layers.LeakyReLU()(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), padding="same")(y)
decoder_outputs = tf.math.reduce_sum(y, axis = -1)
decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 262144)            786432    
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 64, 64, 64)        12352     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 64)        12352     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 64, 64, 64)        0   

In [8]:
outputs = decoder(z)
vae = tf.keras.Model(inputs=encoder_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

In [20]:
# Loss and optimizer.
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# Configure the model for training.
vae.compile(optimizer, loss=loss_fn)

## Normal Autoencoder Architecture

In [14]:
encoder_input = tf.keras.Input(shape = (100, 100, 1))

x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(encoder_input)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((3, 1))(x)
x = tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same')(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.MaxPooling2D((3, 1))(x)
x = tf.keras.layers.Dense(32)(x)
x = tf.keras.layers.Flatten()(x)
encoder_output = tf.keras.layers.Dense(16)(x)

encoder = tf.keras.Model(inputs = encoder_input, outputs = encoder_output, name = "encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 100, 100, 64)      448       
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 100, 100, 64)      24640     
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 100, 100, 64)      24640     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 100, 100, 64)      0   

In [15]:
decoder_input = tf.keras.Input(shape=(16,))
y = tf.keras.layers.Dense(100*100*64)(decoder_input)
y = layers.Reshape((100, 100, 64))(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), activation="relu", padding="same")(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), activation="relu", padding="same")(y)
y = layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), activation="sigmoid", padding="same")(y)
decoder_output = tf.math.reduce_sum(y, axis = -1)

decoder = tf.keras.Model(inputs = decoder_input, outputs = decoder_output, name = "decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 16)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 640000)            10880000  
_________________________________________________________________
reshape_2 (Reshape)          (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 100, 100, 64)      12352     
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 100, 100, 64)      12352     
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 100, 100, 64)      12352     
_________________________________________________________________
tf.math.reduce_sum_2 (TFOpLa (None, 100, 100)          0   

## Class Version

In [20]:
class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([tf.keras.layers.Input(shape = (100, 100, 1)),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Dense(32),
                                            tf.keras.layers.Flatten(),
                                            tf.keras.layers.Dense(16)
                                            
    ])
        self.decoder = tf.keras.Sequential([tf.keras.layers.Input(shape=(16,)),
                                            tf.keras.layers.Dense(100*100*64),
                                            tf.keras.layers.Reshape((100, 100, 64)),
                                            tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), padding="same"),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), padding="same"),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(1, 3), strides=(1, 1), activation="sigmoid", padding="same")
    
    ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return tf.math.reduce_sum(decoded, -1)

In [21]:
latent_dim = 2
autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

## Classification (2) & Regression (1) Model

In [5]:
class Supervised(Model):
    def __init__(self, last_unit):
        super(Supervised, self).__init__()
        self.last_unit = last_unit   
        self.arch = tf.keras.Sequential([tf.keras.layers.Input(shape = (100, 100, 1)),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Conv2D(32, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(32, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(64, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Conv2D(32, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(32, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D((3, 1)),
                                            tf.keras.layers.Conv2D(16, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Conv2D(16, (2, 3), (1, 1), padding = 'same'),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.Flatten(),
                                            tf.keras.layers.Dense(128),
                                            tf.keras.layers.Dense(64),
                                            tf.keras.layers.Dense(32),
                                            tf.keras.layers.Dense(last_unit)])
                                            

    def call(self, x):
        out = self.arch(x)
        return out

In [10]:
last_unit = 2
arch = Supervised(last_unit)
arch.compile(optimizer='adam', loss=losses.BinaryCrossentropy())